## Create SparkSession

In [7]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("WeatherAnalysis").getOrCreate()

+---------+
|namespace|
+---------+
|  default|
|     mydb|
+---------+



## Show databases from HIVE

In [10]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
|     mydb|
+---------+



## Use mydb database

In [11]:
spark.sql('use mydb')

DataFrame[]

## Show mydb tables

In [12]:
spark.sql('show tables').show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
|    mydb|             authors|      false|
|    mydb|                cars|      false|
|    mydb|weather_data_part...|      false|
+--------+--------------------+-----------+



In [ ]:
spark.sql()